In [51]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import ensemble
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import re
import nltk
from wordcloud import WordCloud
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time 
import contractions
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.linear_model import LogisticRegression
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing  import LabelEncoder



### Importing of Kaggle Repository Twitter Dataset

In this Classification Task, we are trying to train a model to classify our crawled dataset whether the tweets are subjective or objective, positive or negative sentiments

In [2]:
tweets = pd.read_csv("textblob_train.csv")

In [3]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88045 entries, 0 to 88044
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             88045 non-null  int64  
 1   tweet_id               88045 non-null  float64
 2   ticker_symbol          88045 non-null  object 
 3   writer                 88045 non-null  object 
 4   post_date              88045 non-null  object 
 5   body                   88045 non-null  object 
 6   comment_num            88045 non-null  int64  
 7   retweet_num            88045 non-null  int64  
 8   like_num               88045 non-null  int64  
 9   tweet_activity         88045 non-null  int64  
 10  sentiment_category     88045 non-null  int64  
 11  subjectivity_category  88045 non-null  int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 8.1+ MB


In [4]:
tweets.head()

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0


In [5]:
tweets.head()

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0


### Importing of Crawled Twitter Dataset

This is the crawled dataset from twitter with regards to stocks such as AAPL, AMZN, MSFT, TSLA, GOOGL

In [6]:
crawled_tweets = pd.read_csv('new_label_acc_100.csv')

In [7]:
crawled_tweets.tail()

,ticker_symbol,post_date,tweet_id,writer,body,like_num,comment_num,retweet_num,tweet_activity,url_cnt,...,word_count,clean_text,subjectivity_1,sentiment_1,subjectivity_2,sentiment_2,moderator_sub,moderator_sen,vote_subjectivity,vote_sentiment
4293,TSLA,2023-02-28 23:25:43+00:00,1.630711e+18,BestTrader01,"$TSLA #TSLA Options Interest! $800sssss, 600ss...",59,7,5,71,1,...,10,tsla tsla option interest 800sssss 600sss 300s...,0,0,0,0,NaN,NaN,0.0,0.0
4294,TSLA,2023-02-28 23:23:40+00:00,1.630710e+18,MatchasmMatt,$RIVN down nearly 10% after earnings. These co...,84,10,2,96,0,...,10,rivn nearli 10 earn compani make tsla look goo...,1,1,1,1,NaN,NaN,1.0,1.0
4295,TSLA,2023-02-28 23:20:10+00:00,1.630709e+18,susanblas,No surprise here. $TSLA does not give a flying...,39,0,11,50,0,...,8,surpris tsla give fli fuck custom theyv taken,1,-1,1,-1,NaN,NaN,1.0,-1.0
4296,TSLA,2023-02-28 23:12:27+00:00,1.630708e+18,stevenmarkryan,Is it just me or is it CREEPY AF that I get no...,267,37,7,311,0,...,23,creepi af get notif someon add list includ ad ...,1,1,0,0,0.0,0.0,0.0,0.0
4297,TSLA,2023-02-28 23:06:48+00:00,1.630706e+18,stevenmarkryan,Can't wait to see how pissed the cry babies wh...,289,16,15,320,0,...,31,cant wait see piss cri babi also tsla elonmusk...,1,-1,1,-1,NaN,NaN,1.0,-1.0


In [8]:
crawled_tweets['vote_subjectivity'] = crawled_tweets['vote_subjectivity'].astype(int)
crawled_tweets['vote_sentiment']= crawled_tweets['vote_sentiment'].astype(int)

In [9]:
crawled_tweets

,ticker_symbol,post_date,tweet_id,writer,body,like_num,comment_num,retweet_num,tweet_activity,url_cnt,...,word_count,clean_text,subjectivity_1,sentiment_1,subjectivity_2,sentiment_2,moderator_sub,moderator_sen,vote_subjectivity,vote_sentiment
0,AAPL,2023-01-01 22:23:31+00:00,1.609677e+18,David_Tracey,Things are going to get very interesting but w...,27,1,2,30,2,...,11,thing go get interest q1 forecast updat aapl r...,0,0,0,0,NaN,NaN,0,0
1,AAPL,2023-01-01 21:04:45+00:00,1.609657e+18,ASTS_Investors,Happy New Year SpaceMob! \n\nLet's hope 2023 w...,64,0,3,67,1,...,21,happi new year spacemob let hope 2023 year blu...,0,1,0,1,NaN,NaN,0,1
2,AAPL,2023-01-01 20:44:51+00:00,1.609652e+18,BigCheds,$AAPL Apple weekly chart - hammer candle with ...,104,9,5,118,1,...,10,aapl appl weekli chart hammer candl spring jun...,0,0,0,0,NaN,NaN,0,0
3,AAPL,2023-01-01 19:45:30+00:00,1.609637e+18,ThetaWarrior,$AAPL Yearly -26.8% https://t.co/SewErQmOPE,80,10,10,100,1,...,4,aapl yearli 268 urlplacehold,0,0,0,0,NaN,NaN,0,0
4,AAPL,2023-01-01 19:00:00+00:00,1.609626e+18,TrendSpider,Looking for shorts into the new year? 📉\n\nThi...,28,4,4,36,1,...,23,look short new year emojiplacehold weekli tria...,0,1,0,1,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4293,TSLA,2023-02-28 23:25:43+00:00,1.630711e+18,BestTrader01,"$TSLA #TSLA Options Interest! $800sssss, 600ss...",59,7,5,71,1,...,10,tsla tsla option interest 800sssss 600sss 300s...,0,0,0,0,NaN,NaN,0,0
4294,TSLA,2023-02-28 23:23:40+00:00,1.630710e+18,MatchasmMatt,$RIVN down nearly 10% after earnings. These co...,84,10,2,96,0,...,10,rivn nearli 10 earn compani make tsla look goo...,1,1,1,1,NaN,NaN,1,1
4295,TSLA,2023-02-28 23:20:10+00:00,1.630709e+18,susanblas,No surprise here. $TSLA does not give a flying...,39,0,11,50,0,...,8,surpris tsla give fli fuck custom theyv taken,1,-1,1,-1,NaN,NaN,1,-1
4296,TSLA,2023-02-28 23:12:27+00:00,1.630708e+18,stevenmarkryan,Is it just me or is it CREEPY AF that I get no...,267,37,7,311,0,...,23,creepi af get notif someon add list includ ad ...,1,1,0,0,0.0,0.0,0,0


In [10]:
crawled_tweets.head()

,ticker_symbol,post_date,tweet_id,writer,body,like_num,comment_num,retweet_num,tweet_activity,url_cnt,...,word_count,clean_text,subjectivity_1,sentiment_1,subjectivity_2,sentiment_2,moderator_sub,moderator_sen,vote_subjectivity,vote_sentiment
0,AAPL,2023-01-01 22:23:31+00:00,1.609677e+18,David_Tracey,Things are going to get very interesting but w...,27,1,2,30,2,...,11,thing go get interest q1 forecast updat aapl r...,0,0,0,0,NaN,NaN,0,0
1,AAPL,2023-01-01 21:04:45+00:00,1.609657e+18,ASTS_Investors,Happy New Year SpaceMob! \n\nLet's hope 2023 w...,64,0,3,67,1,...,21,happi new year spacemob let hope 2023 year blu...,0,1,0,1,NaN,NaN,0,1
2,AAPL,2023-01-01 20:44:51+00:00,1.609652e+18,BigCheds,$AAPL Apple weekly chart - hammer candle with ...,104,9,5,118,1,...,10,aapl appl weekli chart hammer candl spring jun...,0,0,0,0,NaN,NaN,0,0
3,AAPL,2023-01-01 19:45:30+00:00,1.609637e+18,ThetaWarrior,$AAPL Yearly -26.8% https://t.co/SewErQmOPE,80,10,10,100,1,...,4,aapl yearli 268 urlplacehold,0,0,0,0,NaN,NaN,0,0
4,AAPL,2023-01-01 19:00:00+00:00,1.609626e+18,TrendSpider,Looking for shorts into the new year? 📉\n\nThi...,28,4,4,36,1,...,23,look short new year emojiplacehold weekli tria...,0,1,0,1,NaN,NaN,0,1


After Merging Remove the dataframes that columns are empty

In [11]:
tweets.dropna(inplace = True)

Refactoring the Polarity Values of testing crawled_tweets

Convert our tweet date_time into just date

In [12]:

tweets.head()

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0


In [13]:
crawled_tweets['post_date'] = pd.to_datetime(crawled_tweets['post_date'])
crawled_tweets['post_date'] = crawled_tweets['post_date'].dt.date

We need to differentiate active tweets which are more "Reputable" compared to botting tweets or spams. <br>
Activities can be defined as comments/replies, retweets, and likes.

In [14]:
tweets.sort_values(by = "comment_num" , ascending  = False, inplace = True)
tweets


,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
81081,4180147,1.140000e+18,TSLA,Dope007,11/6/2019,Fraudulent slip there Musk. Convince investors...,0,3,31,34,-1,1
81082,4176267,1.140000e+18,TSLA,TommyThornton,6/6/2019,From the lows your beloved $TSLA only needs to...,0,0,33,33,1,1
81083,4176276,1.140000e+18,TSLA,PTBarnu88732123,6/6/2019,Only a total dipshit would compare $tsla stock...,0,2,27,29,1,1
81084,4176239,1.140000e+18,TSLA,EthicsGradient,6/6/2019,$TSLA Very big news for China!,0,9,43,52,0,0


The tweet activities will be defined by the summation of all comments/replies, likes and retweets to differentiate from bots or spams, where tweets are more "usable" for sentimental analysis

In [15]:
tweets["tweet_activity"] = tweets["comment_num"] + tweets["like_num"] + tweets["retweet_num"]
tweets

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
81081,4180147,1.140000e+18,TSLA,Dope007,11/6/2019,Fraudulent slip there Musk. Convince investors...,0,3,31,34,-1,1
81082,4176267,1.140000e+18,TSLA,TommyThornton,6/6/2019,From the lows your beloved $TSLA only needs to...,0,0,33,33,1,1
81083,4176276,1.140000e+18,TSLA,PTBarnu88732123,6/6/2019,Only a total dipshit would compare $tsla stock...,0,2,27,29,1,1
81084,4176239,1.140000e+18,TSLA,EthicsGradient,6/6/2019,$TSLA Very big news for China!,0,9,43,52,0,0


### Filter Tweets that are >= 25 activities

In [16]:
filterTweets = tweets.loc[tweets["tweet_activity"] > 25]

In [17]:
filterTweets.head()

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0


In [18]:
filterTweets.describe()

,Unnamed: 0,tweet_id,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category
count,8.804500e+04,8.804500e+04,88045.000000,88045.000000,88045.000000,88045.000000,88045.000000,88045.000000
mean,3.266381e+06,1.037548e+18,6.009790,17.359135,58.161804,81.530729,0.285763,0.315566
std,1.165772e+06,1.434625e+17,11.197338,45.525474,75.893739,104.740242,0.773832,0.464743
min,4.000000e+01,5.500000e+17,0.000000,0.000000,0.000000,26.000000,-1.000000,0.000000
25%,2.490996e+06,9.960000e+17,1.000000,4.000000,24.000000,34.000000,0.000000,0.000000
50%,3.867796e+06,1.070000e+18,3.000000,8.000000,35.000000,48.000000,0.000000,0.000000
75%,4.112427e+06,1.130000e+18,7.000000,17.000000,61.000000,84.000000,1.000000,1.000000
max,4.336399e+06,1.210000e+18,631.000000,999.000000,999.000000,1703.000000,1.000000,1.000000


### Data Cleaning and Tokenization of text body

In [19]:
# Function to remove stopwords (common english stopwords from nltk)
def removeStopWords(df):
    finalList = []
    stop_words = set(stopwords.words('english'))
    negativeList = ['not','no', 'nor']
    for x in negativeList:
        stop_words.remove(x)
    for word in df:
        if word not in stop_words:
            finalList.append(word)
    df = finalList
    return df

In [20]:
# Function to stem tweets 
def stem_text(text):
    
    stemmer = PorterStemmer()
   
    return [stemmer.stem(word) for word in text]

In [21]:
# Function to lemmatise tweets
def lemmatised_text(text):
    lemmatiser = nltk.stem.WordNetLemmatizer()
    return [lemmatiser.lemmatize(word) for word in text]

In [22]:
def replace_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

In [23]:
def remove_username(text):
    return re.sub(r'@\w+', '', text)

In [24]:
def remove_hashtag(text):
    return re.sub(r'#\w+', '', text)

In [25]:
def clean_dataframe_text(df, column,lemma):
    
    # Define regex pattern to match URLs
    url_pattern = r"https?://\S+"
    # Remove URLs from text column
    df[column] = df[column].str.replace(url_pattern, "", regex=True)
    # Remove username from text column
    df[column] = df[column].apply(lambda x: remove_username(x))
     # Remove hashtag from text column
    df[column] = df[column].apply(lambda x: remove_hashtag(x))
    # Replace Contractions
    df[column] = df[column].apply(lambda x: replace_contractions(x))
    # Remove any non-alphanumeric characters and replace them with spaces
    df[column] = df[column].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    # Replace any consecutive whitespace characters with a single space
    df[column] = df[column].apply(lambda x: re.sub(r'\s+', ' ', x))
    # Remove not word characters
    df[column] = df[column].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
    # Remove digits 
    df[column] = df[column].apply(lambda x: re.sub(r'\d+', '', x))
    # Convert the string in lower
    df[column] = df[column].str.lower()
    # Tokenised the words
    df[column]  = df[column].apply(lambda x : nltk.word_tokenize(x))
    # Remove stopwords from the NLTK stopword list
    df[column] = df[column].apply(lambda x : removeStopWords(x))
    if lemma:
    # lemmatised the text
        df[column] = df[column].apply(lambda x : lemmatised_text(x))
    else:
    # Stemmed the text
        df[column] = df[column].apply(lambda x : stem_text(x))
    #concatenate the tokenised list into string
    df[column] = df[column].apply(lambda x: ' '.join(x))
    return df

In [26]:
# Tweets that are cleaned and stemmed
filterTweets['clean_text'] = filterTweets['body']
filterTweets = clean_dataframe_text(filterTweets,"clean_text",True)
filterTweets.head()

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category,clean_text
0,3786974,1.010000e+18,TSLA,lopezlinette,5/7/2018,"I’d just like to point out that right now, rig...",631,369,0,1000,1,0,would like point right right moment going face...
1,4112626,1.120000e+18,TSLA,GerberKawasaki,18/4/2019,"If a Tesla saves you $200 a month on gas, you ...",567,0,0,567,1,0,tesla save month gas ten year take saving put ...
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1,tesla feature request threadfeel free reply co...
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,-1,1,sell book versus sell whatever fuck want amzn
4,4182686,1.140000e+18,TSLA,TeslaNY,13/6/2019,“I think it's basically financially insane to ...,442,0,0,442,-1,0,think basically financially insane buy anythin...


In [27]:
crawled_tweets['clean_text'] = crawled_tweets['body']

In [28]:

crawled_tweets = clean_dataframe_text(crawled_tweets,"clean_text",True)

### Convert Positive, Negative and Neutral to numeral labels and Subjectivity

### Splitting up data for Prediction using ensemble machine learning models For Blob Subjectivity and Polarity Analysis

In [29]:
X = filterTweets["clean_text"].values 
y = filterTweets[["sentiment_category"]].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [30]:
X_subj = filterTweets["clean_text"].values 
y_subj = filterTweets["subjectivity_category"].values
X_train_subj, X_test_subj, y_train_subj, y_test_subj = train_test_split(
    X_subj, y_subj, test_size=0.20, random_state=42)

In [31]:
filterTweets["subjectivity_category"].value_counts()

0    60261
1    27784
Name: subjectivity_category, dtype: int64

### Evaluation Data -> Crawled and labelled data

In [32]:
#Evluation Data for Subjectivity
eval_data_x = crawled_tweets['clean_text'].values
eval_data_y = crawled_tweets['vote_subjectivity'].values


In [33]:
def metric_report(y_test,y_pred,time,no_record_x_test,y_eval, y_pred_eval,time_eval,no_record_x_eval):
    accuracy = accuracy_score(y_test,y_pred)
    fmeasure = f1_score(y_test,y_pred, average = "weighted", zero_division = 1)
    recall = recall_score(y_test, y_pred, average = "weighted",zero_division = 1)
    precision = precision_score(y_test, y_pred, average = "weighted",zero_division = 1)

    
    print(f"Model Metrics with testing data")
    print(f"The accuracy of the Model is {accuracy}")
    print(f"The F-Score of the Model is {fmeasure}")
    print(f"The Recall of the Model is {recall}")
    print(f"The precision of the Model is {precision}")
    print(f"The time taken for the Model prediction is {time} seconds")
    print(f"The number of records per second is {no_record_x_test//time}")
    print(f"================================================================")
    

    accuracy_eval = accuracy_score(y_eval,y_pred_eval)
    fmeasure_eval = f1_score(y_eval,y_pred_eval, average = "weighted", zero_division = 1)
    recall_eval = recall_score(y_eval, y_pred_eval, average = "weighted",zero_division = 1)
    precision_eval = precision_score(y_eval, y_pred_eval, average = "weighted",zero_division = 1)
    print(f"Model Metrics with evaluation data")
    print(f"The accuracy of the Model is {accuracy_eval}")
    print(f"The F-Score of the Model is {fmeasure_eval}")
    print(f"The Recall of the Model is {recall_eval}")
    print(f"The precision of the Model is {precision_eval}")
    print(f"The time taken for the Model prediction is {time_eval} seconds")
    print(f"The number of records per second is {no_record_x_eval//time_eval}")

In [34]:
def metric_report_noeval(y_test,y_pred,time,no_record_x_test):
    accuracy = accuracy_score(y_test,y_pred)
    fmeasure = f1_score(y_test,y_pred, average = "weighted", zero_division = 1)
    recall = recall_score(y_test, y_pred, average = "weighted",zero_division = 1)
    precision = precision_score(y_test, y_pred, average = "weighted",zero_division = 1)

    
    print(f"Model Metrics with testing data")
    print(f"The accuracy of the Model is {accuracy}")
    print(f"The F-Score of the Model is {fmeasure}")
    print(f"The Recall of the Model is {recall}")
    print(f"The precision of the Model is {precision}")
    print(f"The time taken for the Model prediction is {time} seconds")
    print(f"The number of records per second is {no_record_x_test//time}")
    print(f"================================================================")

### Vectorise and TFIDF train and test datas

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [36]:
# Create Count Vectorizer (TF)
vectoriser = CountVectorizer()

# For blob analysis dataset that have been lemmatised

X_train_vec = vectoriser.fit_transform(X_train)
X_test_vec = vectoriser.transform(X_test)
eval_data_x_vec = vectoriser.transform(eval_data_x)


# For Subjectivity Blob Analysis Vectorised
X_train_subj_vec = vectoriser.fit_transform(X_train_subj)
X_test_subj_vec = vectoriser.transform(X_test_subj)
eval_data_x_subj_vec = vectoriser.transform(eval_data_x)


# Create TFID Vectorizer 
vectoriser_tfidf = TfidfVectorizer()
# For blob analysis dataset that have been lemmatised


X_train_tfidf = vectoriser_tfidf.fit_transform(X_train)
X_test_tfidf = vectoriser_tfidf.transform(X_test)
eval_data_x_tfidf = vectoriser_tfidf.transform(eval_data_x)

# For Subjectivity Blob Analysis TFIDF
X_train_subj_tfidf = vectoriser_tfidf.fit_transform(X_train_subj)
X_test_subj_tfidf = vectoriser_tfidf.transform(X_test_subj)
eval_data_x_subj_tfidf = vectoriser_tfidf.transform(eval_data_x)




In [37]:
filterSubjTweets = filterTweets.copy()
filterSubjTweets = filterSubjTweets.loc[filterSubjTweets['subjectivity_category'] == 1]



In [38]:
filterSubjTweets = filterSubjTweets.loc[filterSubjTweets['sentiment_category'] != 0]


In [39]:
sentiment_map_self = {-1 : 0}

In [40]:
filterSubjTweets["sentiment_category"].replace(sentiment_map_self, inplace = True)


In [41]:
filterSubjTweets

,Unnamed: 0,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,tweet_activity,sentiment_category,subjectivity_category,clean_text
2,4306536,1.200000e+18,TSLA,AlexSibila,26/11/2019,~Tesla feature request thread~Feel free to rep...,563,48,662,1273,1,1,tesla feature request threadfeel free reply co...
3,2823018,8.860000e+17,AMZN,internet_dust,14/7/2017,"""I sell books."" versus ""I sell whatever the fu...",496,0,0,496,0,1,sell book versus sell whatever fuck want amzn
11,4276279,1.190000e+18,TSLA,jack,24/10/2019,Now anyone can buy $42 or even $1 worth of Ber...,298,589,0,887,1,1,anyone buy even worth berkshire hathaway brka ...
12,4076527,1.110000e+18,TSLA,ElectricTempus,17/3/2019,"People are buying Tesla’s, in their Tesla’s. ...",292,0,0,292,1,1,people buying tesla tesla many reason love com...
13,3750143,1.000000e+18,TSLA,MattLevinson,25/5/2018,Thank you @elonmusk and @tesla for by FAR the ...,290,840,0,1130,1,1,thank far best consumer product ever used day ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81079,4179793,1.140000e+18,TSLA,Polixenes13,11/6/2019,$TSLA's Nevada Gigafactory is a fine place for...,0,7,33,40,0,1,tslas nevada gigafactory fine place making cyl...
81080,4180250,1.140000e+18,TSLA,PlugInFUD,11/6/2019,"""this is quite a hard problem. ignore that we ...",0,4,26,30,1,1,quite hard problem ignore completely fabricate...
81081,4180147,1.140000e+18,TSLA,Dope007,11/6/2019,Fraudulent slip there Musk. Convince investors...,0,3,31,34,0,1,fraudulent slip musk convince investor oops me...
81082,4176267,1.140000e+18,TSLA,TommyThornton,6/6/2019,From the lows your beloved $TSLA only needs to...,0,0,33,33,1,1,low beloved tsla need go get old high


In [42]:
X = filterSubjTweets["clean_text"].values 
y = filterSubjTweets["sentiment_category"].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=0)

In [43]:
crawled_tweets = crawled_tweets.loc[crawled_tweets['vote_subjectivity'] != 0]
crawled_tweets = crawled_tweets.loc[crawled_tweets['vote_sentiment'] != 0]


In [44]:
crawled_tweets["vote_sentiment"].replace(sentiment_map_self, inplace = True)


In [45]:
#Evluation Data for Subjectivity
eval_data_x = crawled_tweets['clean_text'].values
eval_data_y_polarity = crawled_tweets['vote_sentiment'].values


In [46]:
# Create Count Vectorizer (TF)
vectoriser = CountVectorizer()

# For blob analysis dataset that have been lemmatised

X_train_vec = vectoriser.fit_transform(X_train)
X_test_vec = vectoriser.transform(X_test)
eval_data_x_vec = vectoriser.transform(eval_data_x)


# Create TFID Vectorizer 
vectoriser_tfidf = TfidfVectorizer()
# For blob analysis dataset that have been lemmatised

X_train_tfidf = vectoriser_tfidf.fit_transform(X_train)
X_test_tfidf = vectoriser_tfidf.transform(X_test)
eval_data_x_tfidf = vectoriser_tfidf.transform(eval_data_x)







In [47]:
loss = "binary_crossentropy"

### Ensemble Stacked for NN MLP SVC and LR

In [48]:
from scikeras.wrappers import KerasClassifier

In [49]:
def create_model(X_shape):
    
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (X_shape.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

    # Compile the model_tfidf
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
    model._estimator_type = 'classifier'
    return model

In [64]:
# Start Training with Count Vectoriser

from mlxtend.classifier import StackingClassifier

# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = MLPClassifier(solver = "adam", alpha = 1e-5, 
                   hidden_layer_sizes =(64,2), activation = 'tanh',  max_iter = 1000, early_stopping = True)

# Use these models to make predictions on your data
lr.fit(X_train_vec,y_train)
svm.fit(X_train_vec,y_train)
nn.fit(X_train_vec,y_train)

meta_model = LogisticRegression()
stacking_model = StackingClassifier(classifiers=[lr, svm, nn], meta_classifier=meta_model)
stacking_model.fit(X_train_vec, y_train)
start_time = time.time()
y_pred = stacking_model.predict(X_test_vec)
end_time = time.time()
y_pred = (y_pred >= 0.5).astype(int)


start_time_eval = time.time()
y_pred_eval = stacking_model.predict(eval_data_x_vec)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Model Metrics with testing data
The accuracy of the Model is 0.892774343122102
The F-Score of the Model is 0.8914227310463367
The Recall of the Model is 0.892774343122102
The precision of the Model is 0.8911691169882233
The time taken for the Model prediction is 11.35392689704895 seconds
The number of records per second is 455.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8574821852731591
The F-Score of the Model is 0.8490992368056043
The Recall of the Model is 0.8574821852731591
The precision of the Model is 0.8568287436770348
The time taken for the Model prediction is 1.5064880847930908 seconds
The number of records per second is 558.0


In [ ]:
# Start Training with tfidf_vectorizer

from mlxtend.classifier import StackingClassifier

# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = MLPClassifier(solver = "adam", alpha = 1e-5, 
                    hidden_layer_sizes =(64,2), activation = 'tanh',  max_iter = 1000, early_stopping = True)

# Use these models to make predictions on your data
lr.fit(X_train_tfidf,y_train)
svm.fit(X_train_tfidf,y_train)
nn.fit(X_train_tfidf,y_train)

meta_model = LogisticRegression()
stacking_model = StackingClassifier(classifiers=[lr, svm, nn], meta_classifier=meta_model)
stacking_model.fit(X_train_tfidf, y_train)
start_time = time.time()
y_pred = stacking_model.predict(X_test_tfidf)
end_time = time.time()
y_pred = (y_pred >= 0.5).astype(int)
time_taken = end_time - start_time

start_time_eval = time.time()
y_pred_eval = stacking_model.predict(eval_data_x_tfidf)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Model Metrics with testing data
The accuracy of the Model is 0.874613601236476
The F-Score of the Model is 0.86827084864194
The Recall of the Model is 0.874613601236476
The precision of the Model is 0.8754627054698726
The time taken for the Model prediction is 0.5272204875946045 seconds
The number of records per second is 9817.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8479809976247031
The F-Score of the Model is 0.8339784532507322
The Recall of the Model is 0.8479809976247031
The precision of the Model is 0.854245758640058
The time taken for the Model prediction is 2.2750065326690674 seconds
The number of records per second is 370.0


In [66]:
from sklearn.ensemble import StackingClassifier
# Start Training with tfidf_vectorizer

# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = KerasClassifier(model= create_model(X_train_vec), epochs=5, batch_size=32)

# Use these models to make predictions on your data
lr.fit(X_train_vec,y_train)
svm.fit(X_train_vec,y_train)
nn.fit(X_train_vec,y_train)

meta_model = LogisticRegression()
stacking_model = StackingClassifier(
    estimators=[('lr', lr), ('svm', svm), ('nn', nn)],
    final_estimator=LogisticRegression()
)
stacking_model.fit(X_train_vec, y_train)
start_time = time.time()
y_pred = stacking_model.predict(X_test_vec)
end_time = time.time()
y_pred = (y_pred >= 0.5).astype(int)
time_taken = end_time - start_time


start_time_eval = time.time()
y_pred_eval = stacking_model.predict(eval_data_x_vec)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Epoch 1/5
647/647 [==============================] - 21s 31ms/step - loss: 0.5373 - accuracy: 0.8014
Epoch 2/5
647/647 [==============================] - 23s 35ms/step - loss: 0.3829 - accuracy: 0.9058
Epoch 3/5
647/647 [==============================] - 23s 36ms/step - loss: 0.3127 - accuracy: 0.9393
Epoch 4/5
647/647 [==============================] - 24s 37ms/step - loss: 0.2776 - accuracy: 0.9531
Epoch 5/5
647/647 [==============================] - 23s 36ms/step - loss: 0.2581 - accuracy: 0.9610


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpt5lv0awn\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpt5lv0awn\assets


Epoch 1/5
647/647 [==============================] - 21s 32ms/step - loss: 0.2463 - accuracy: 0.9637
Epoch 2/5
647/647 [==============================] - 21s 33ms/step - loss: 0.2458 - accuracy: 0.9640
Epoch 3/5
647/647 [==============================] - 20s 31ms/step - loss: 0.2352 - accuracy: 0.9680
Epoch 4/5
647/647 [==============================] - 20s 32ms/step - loss: 0.2354 - accuracy: 0.9680
Epoch 5/5
647/647 [==============================] - 22s 34ms/step - loss: 0.2290 - accuracy: 0.9705


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmph98o2ukg\assets


...

INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpdrqfmav5\assets


Epoch 1/5
518/518 [==============================] - 18s 33ms/step - loss: 0.2426 - accuracy: 0.9658
Epoch 2/5
518/518 [==============================] - 17s 33ms/step - loss: 0.2301 - accuracy: 0.9691
Epoch 3/5
518/518 [==============================] - 17s 33ms/step - loss: 0.2210 - accuracy: 0.9723
Epoch 4/5
518/518 [==============================] - 17s 33ms/step - loss: 0.2119 - accuracy: 0.9743
Epoch 5/5
130/130 [==============================] - 0s 945us/step


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpknkfn89p\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpknkfn89p\assets


Epoch 1/5
518/518 [==============================] - 18s 34ms/step - loss: 0.2416 - accuracy: 0.9649
Epoch 2/5
518/518 [==============================] - 17s 33ms/step - loss: 0.2362 - accuracy: 0.9669
Epoch 3/5
518/518 [==============================] - 17s 33ms/step - loss: 0.2183 - accuracy: 0.9739
Epoch 4/5
518/518 [==============================] - 18s 34ms/step - loss: 0.2108 - accuracy: 0.9750
Epoch 5/5
27/27 [==============================] - 0s 997us/step
Model Metrics with testing data
The accuracy of the Model is 0.8904559505409583
The F-Score of the Model is 0.8895946101430124
The Recall of the Model is 0.8904559505409583
The precision of the Model is 0.8891756693421495
The time taken for the Model prediction is 10.612773656845093 seconds
The number of records per second is 487.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8681710213776722
The F-Score of the Model is 0.8613834700506667
The Recall of the Model is 0.8681710213776722
The precision of the 

In [65]:
# Start Training with tfidf_vectorizer
from sklearn.ensemble import StackingClassifier

# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = KerasClassifier(model= create_model(X_train_tfidf) , epochs=5, batch_size=32)


meta_model = LogisticRegression()
stacking_model = StackingClassifier(
    estimators=[('lr', lr), ('svm', svm), ('nn', nn)],
    final_estimator=LogisticRegression()
)

stacking_model.fit(X_train_tfidf, y_train)
start_time = time.time()
y_pred = stacking_model.predict(X_test_tfidf)
end_time = time.time()
y_pred = (y_pred >= 0.5).astype(int)
time_taken = end_time - start_time


start_time_eval = time.time()
y_pred_eval = stacking_model.predict(eval_data_x_tfidf)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpprbq3l84\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpprbq3l84\assets


Epoch 1/5
647/647 [==============================] - 23s 35ms/step - loss: 0.5233 - accuracy: 0.7919
Epoch 2/5
647/647 [==============================] - 20s 30ms/step - loss: 0.3677 - accuracy: 0.8979
Epoch 3/5
647/647 [==============================] - 18s 28ms/step - loss: 0.3050 - accuracy: 0.9340
Epoch 4/5
647/647 [==============================] - 18s 28ms/step - loss: 0.2683 - accuracy: 0.9488
Epoch 5/5
647/647 [==============================] - 19s 30ms/step - loss: 0.2420 - accuracy: 0.9598


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpbz762w0u\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpbz762w0u\assets


...

INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpu298wqiz\assets


Epoch 1/5
518/518 [==============================] - 21s 38ms/step - loss: 0.5511 - accuracy: 0.7726
Epoch 2/5
518/518 [==============================] - 18s 35ms/step - loss: 0.4146 - accuracy: 0.8920
Epoch 3/5
518/518 [==============================] - 18s 35ms/step - loss: 0.3513 - accuracy: 0.9268
Epoch 4/5
518/518 [==============================] - 18s 35ms/step - loss: 0.3146 - accuracy: 0.9422
Epoch 5/5
130/130 [==============================] - 0s 978us/step


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpjnxw0t65\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpjnxw0t65\assets


Epoch 1/5
518/518 [==============================] - 18s 34ms/step - loss: 0.5503 - accuracy: 0.7727
Epoch 2/5
518/518 [==============================] - 17s 33ms/step - loss: 0.4071 - accuracy: 0.8892
Epoch 3/5
518/518 [==============================] - 17s 33ms/step - loss: 0.3305 - accuracy: 0.9279
Epoch 4/5
518/518 [==============================] - 17s 33ms/step - loss: 0.2916 - accuracy: 0.9456
Epoch 5/5
27/27 [==============================] - 0s 1ms/step
Model Metrics with testing data
The accuracy of the Model is 0.892774343122102
The F-Score of the Model is 0.8924357089269304
The Recall of the Model is 0.892774343122102
The precision of the Model is 0.8921718085861989
The time taken for the Model prediction is 11.44518256187439 seconds
The number of records per second is 452.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8741092636579573
The F-Score of the Model is 0.8680090922400242
The Recall of the Model is 0.8741092636579573
The precision of the Model

### Ensemble Voting Classifier

In [122]:
#Ensemble Classifier for Vectorised 
clf1 = LogisticRegression(max_iter = 1000)
clf2 = MLPClassifier(solver = "adam", alpha = 1e-5, 
                   hidden_layer_sizes =(64,2), activation = 'tanh',  max_iter = 1000, early_stopping = True)
clf3 = SVC(kernel = "rbf", random_state = 1)

# Create the ensemble classifier
ensemble = VotingClassifier(estimators=[('lr', clf1), ('nn', clf2), ('svc', clf3)], voting='hard')

# Train the ensemble classifier
ensemble.fit(X_train_vec,y_train)
start_time = time.time()
# Make predictions on the test set
y_pred = ensemble.predict(X_test_vec)
end_time = time.time()
time_taken =  end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

start_time_eval = time.time()
y_pred_eval = ensemble.predict(eval_data_x_vec)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)


# Rounding off time to the nearest 0.00001s to avoid values to 0.
if time_taken <= 0.00001:
    time_taken = 0.00001
    
if time_taken_eval <= 0.00001:
    time_taken_eval = 0.00001
# Evaluate the model using accuracy score

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Model Metrics with testing data
The accuracy of the Model is 0.8887171561051005
The F-Score of the Model is 0.8860434138335391
The Recall of the Model is 0.8887171561051005
The precision of the Model is 0.8870148190622066
The time taken for the Model prediction is 10.120749235153198 seconds
The number of records per second is 511.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8551068883610451
The F-Score of the Model is 0.8446343086790544
The Recall of the Model is 0.8551068883610451
The precision of the Model is 0.857101661891036
The time taken for the Model prediction is 1.5841286182403564 seconds
The number of records per second is 531.0


In [126]:
#Ensemble Classifier for tfidf
clf1 = LogisticRegression(max_iter = 1000)
clf2 = MLPClassifier(solver = "adam", alpha = 1e-5, 
                   hidden_layer_sizes =(64,2), activation = 'tanh',  max_iter = 1000, early_stopping = True)
clf3 = SVC(kernel = "rbf", random_state = 1)

# Create the ensemble classifier
ensemble = VotingClassifier(estimators=[('lr', clf1), ('nn', clf2), ('svc', clf3)], voting='hard')

# Train the ensemble classifier
ensemble.fit(X_train_tfidf,y_train)
start_time = time.time()
# Make predictions on the test set
y_pred = ensemble.predict(X_test_tfidf)
end_time = time.time()
time_taken =  end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

start_time_eval = time.time()
y_pred_eval = ensemble.predict(eval_data_x_tfidf)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)


# Rounding off time to the nearest 0.00001s to avoid values to 0.
if time_taken <= 0.00001:
    time_taken = 0.00001
    
if time_taken_eval <= 0.00001:
    time_taken_eval = 0.00001
# Evaluate the model using accuracy score

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Model Metrics with testing data
The accuracy of the Model is 0.8736476043276662
The F-Score of the Model is 0.8672763540467812
The Recall of the Model is 0.8736476043276662
The precision of the Model is 0.8743353709339633
The time taken for the Model prediction is 10.031439065933228 seconds
The number of records per second is 515.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8479809976247031
The F-Score of the Model is 0.8331694307034736
The Recall of the Model is 0.8479809976247031
The precision of the Model is 0.8561338846534287
The time taken for the Model prediction is 1.6260128021240234 seconds
The number of records per second is 517.0


In [129]:
#Ensemble Classifier for tfidf
clf1 = LogisticRegression(max_iter = 1000)
clf2 = KerasClassifier(model= create_model(X_train_tfidf), epochs=5, batch_size=32)
clf3 = SVC(kernel = "rbf", random_state = 1)

# Create the ensemble classifier
ensemble = VotingClassifier(estimators=[('lr', clf1), ('keras', clf2), ('svc', clf3)], voting='hard')

# Train the ensemble classifier
ensemble.fit(X_train_tfidf,y_train)
start_time = time.time()
# Make predictions on the test set
y_pred = ensemble.predict(X_test_tfidf)
end_time = time.time()
time_taken =  end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

start_time_eval = time.time()
y_pred_eval = ensemble.predict(eval_data_x_tfidf)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)


# Rounding off time to the nearest 0.00001s to avoid values to 0.
if time_taken <= 0.00001:
    time_taken = 0.00001
    
if time_taken_eval <= 0.00001:
    time_taken_eval = 0.00001
# Evaluate the model using accuracy score

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmprbfwlqkm\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmprbfwlqkm\assets


Epoch 1/5
647/647 [==============================] - 22s 34ms/step - loss: 0.5273 - accuracy: 0.7969
Epoch 2/5
647/647 [==============================] - 21s 33ms/step - loss: 0.3747 - accuracy: 0.9030
Epoch 3/5
647/647 [==============================] - 21s 33ms/step - loss: 0.3075 - accuracy: 0.9398
Epoch 4/5
647/647 [==============================] - 21s 33ms/step - loss: 0.2751 - accuracy: 0.9545
Epoch 5/5
27/27 [==============================] - 0s 894us/step
Model Metrics with testing data
The accuracy of the Model is 0.8763523956723338
The F-Score of the Model is 0.8701175330121406
The Recall of the Model is 0.8763523956723338
The precision of the Model is 0.8773925137820683
The time taken for the Model prediction is 11.41761827468872 seconds
The number of records per second is 453.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8467933491686461
The F-Score of the Model is 0.8316585674153436
The Recall of the Model is 0.8467933491686461
The precision of the M

In [130]:
#Ensemble Classifier for vec
clf1 = LogisticRegression(max_iter = 1000)
clf2 = KerasClassifier(model= create_model(X_train_vec), epochs=5, batch_size=32)
clf3 = SVC(kernel = "rbf", random_state = 1)

# Create the ensemble classifier
ensemble = VotingClassifier(estimators=[('lr', clf1), ('tf', clf2), ('svc', clf3)], voting='hard')

# Train the ensemble classifier
ensemble.fit(X_train_vec,y_train)
start_time = time.time()
# Make predictions on the test set
y_pred = ensemble.predict(X_test_vec)
end_time = time.time()
time_taken =  end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

start_time_eval = time.time()
y_pred_eval = ensemble.predict(eval_data_x_vec)
end_time_eval = time.time()
time_taken_eval =  end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

# Rounding off time to the nearest 0.00001s to avoid values to 0.
if time_taken <= 0.00001:
    time_taken = 0.00001
    
if time_taken_eval <= 0.00001:
    time_taken_eval = 0.00001
# Evaluate the model using accuracy score

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpeam_zn2m\assets


INFO:tensorflow:Assets written to: C:\Users\eric2\AppData\Local\Temp\tmpeam_zn2m\assets


Epoch 1/5


647/647 [==============================] - 22s 31ms/step - loss: 0.5405 - accuracy: 0.7952
Epoch 2/5
647/647 [==============================] - 19s 29ms/step - loss: 0.3820 - accuracy: 0.9059
Epoch 3/5
647/647 [==============================] - 19s 30ms/step - loss: 0.3083 - accuracy: 0.9400
Epoch 4/5
647/647 [==============================] - 19s 30ms/step - loss: 0.2760 - accuracy: 0.9539
Epoch 5/5
27/27 [==============================] - 0s 959us/step
Model Metrics with testing data
The accuracy of the Model is 0.8887171561051005
The F-Score of the Model is 0.8855717876765451
The Recall of the Model is 0.8887171561051005
The precision of the Model is 0.8873352774583042
The time taken for the Model prediction is 9.863991260528564 seconds
The number of records per second is 524.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8622327790973872
The F-Score of the Model is 0.8525953833193325
The Recall of the Model is 0.8622327790973872
The precision of the Model is 0.

### Weighted Average Ensemble Classification

In [87]:
# Define Weights for the Classifiers

lr_weight = 0.2
svm_weight = 0.5
nn_weight = 0.3

In [88]:
# Weighted Average Ensemble Classification (TFIDF) for Logistic Regression/SVC/MLP

# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = MLPClassifier(solver = "adam", alpha = 1e-5, 
                    hidden_layer_sizes =(64,2), activation = 'tanh',  max_iter = 1000, early_stopping = True)

# Use these models to make predictions on your data
lr.fit(X_train_tfidf,y_train)
svm.fit(X_train_tfidf,y_train)
nn.fit(X_train_tfidf,y_train)

start_time = time.time()
lr_pred = lr.predict(X_test_tfidf)
svm_pred = svm.predict(X_test_tfidf)
nn_pred = nn.predict(X_test_tfidf)

y_pred = (nn_weight*nn_pred) + (svm_weight*svm_pred) + (lr_weight*lr_pred)
end_time = time.time()
time_taken = end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

start_time_eval = time.time()
lr_pred_eval = lr.predict(eval_data_x_tfidf)
svm_pred_eval = svm.predict(eval_data_x_tfidf)
nn_pred_eval = nn.predict(eval_data_x_tfidf)

y_pred_eval = (nn_weight*nn_pred_eval) + (svm_weight*svm_pred_eval) + (lr_weight*lr_pred_eval)
end_time_eval = time.time()
time_taken_eval = end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Model Metrics with testing data
The accuracy of the Model is 0.8717156105100463
The F-Score of the Model is 0.864658740750019
The Recall of the Model is 0.8717156105100463
The precision of the Model is 0.8731860870827328
The time taken for the Model prediction is 13.628058195114136 seconds
The number of records per second is 379.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8467933491686461
The F-Score of the Model is 0.8320720899890943
The Recall of the Model is 0.8467933491686461
The precision of the Model is 0.854102148956951
The time taken for the Model prediction is 1.9108002185821533 seconds
The number of records per second is 440.0


In [89]:
# Weighted Average Ensemble Classification (CountVectorizer) for Logistic Regression/SVC/MLP

# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = MLPClassifier(solver = "adam", alpha = 1e-5, 
                    hidden_layer_sizes =(64,2), activation = 'tanh',  max_iter = 1000, early_stopping = True)

# Use these models to make predictions on your data
lr.fit(X_train_vec,y_train)
svm.fit(X_train_vec,y_train)
nn.fit(X_train_vec,y_train)

start_time = time.time()
lr_pred = lr.predict(X_test_vec)
svm_pred = svm.predict(X_test_vec)
nn_pred = nn.predict(X_test_vec)

y_pred = (nn_weight*nn_pred) + (svm_weight*svm_pred) + (lr_weight*lr_pred)
end_time = time.time()
time_taken = end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

start_time_eval = time.time()
lr_pred_eval = lr.predict(eval_data_x_vec)
svm_pred_eval = svm.predict(eval_data_x_vec)
nn_pred_eval = nn.predict(eval_data_x_vec)

y_pred_eval = (nn_weight*nn_pred_eval) + (svm_weight*svm_pred_eval) + (lr_weight*lr_pred_eval)
end_time_eval = time.time()
time_taken_eval = end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

Model Metrics with testing data
The accuracy of the Model is 0.8693972179289027
The F-Score of the Model is 0.8616390814745114
The Recall of the Model is 0.8693972179289027
The precision of the Model is 0.8716164145134563
The time taken for the Model prediction is 12.20228385925293 seconds
The number of records per second is 424.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8432304038004751
The F-Score of the Model is 0.8270974973932708
The Recall of the Model is 0.8432304038004751
The precision of the Model is 0.8519320703606944
The time taken for the Model prediction is 2.3207924365997314 seconds
The number of records per second is 362.0


In [109]:
# Weighted Average Ensemble Classification (CountVectorizer) for Logistic Regression/SVC/TF-FFN
# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (X_train_vec.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model_tfidf
nn.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

# Use these models to make predictions on your data
lr.fit(X_train_vec,y_train)
svm.fit(X_train_vec,y_train)
nn.fit(X_train_vec,y_train, epochs = 1)

start_time = time.time()
lr_pred = np.array(lr.predict(X_test_vec))
svm_pred = np.array(svm.predict(X_test_vec))
nn_pred = np.array(nn.predict(X_test_vec))
nn_pred = (nn_pred >= 0.5).astype(int)
nn_pred = nn_pred.ravel()

y_pred= (nn_weight*nn_pred) + (svm_weight*svm_pred) + (lr_weight*lr_pred)
end_time = time.time()
time_taken = end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)

y_pred = y_pred.ravel()

start_time_eval = time.time()
lr_pred_eval = np.array(lr.predict(eval_data_x_vec))
svm_pred_eval = np.array(svm.predict(eval_data_x_vec))
nn_pred_eval = np.array(nn.predict(eval_data_x_vec))
nn_pred_eval = nn_pred_eval.ravel()

y_pred_eval = nn_weight*nn_pred_eval+ svm_weight*svm_pred_eval + lr_weight*lr_pred_eval
end_time_eval = time.time()
time_taken_eval = end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)
y_pred_eval = y_pred_eval.ravel()

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

27/27 [==============================] - 0s 882us/step
Model Metrics with testing data
The accuracy of the Model is 0.8684312210200927
The F-Score of the Model is 0.8607282058985348
The Recall of the Model is 0.8684312210200927
The precision of the Model is 0.8702784899319054
The time taken for the Model prediction is 9.970266819000244 seconds
The number of records per second is 519.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8456057007125891
The F-Score of the Model is 0.8305627030582153
The Recall of the Model is 0.8456057007125891
The precision of the Model is 0.8530394380034049
The time taken for the Model prediction is 1.7275280952453613 seconds
The number of records per second is 487.0


In [110]:
# Weighted Average Ensemble Classification (TFIDF) for Logistic Regression/SVC/TF-FFN
# Train several other models on your data using different algorithms or hyperparameters
lr = LogisticRegression(max_iter = 1000)
svm = SVC(kernel = "rbf", random_state = 1)
nn = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model_tfidf
nn.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

# Use these models to make predictions on your data
lr.fit(X_train_tfidf,y_train)
svm.fit(X_train_tfidf,y_train)
nn.fit(X_train_tfidf,y_train)

start_time = time.time()
lr_pred = lr.predict(X_test_tfidf)
svm_pred = svm.predict(X_test_tfidf)
nn_pred = nn.predict(X_test_tfidf)
nn_pred = (nn_pred >= 0.5).astype(int)
nn_pred = nn_pred.ravel()

y_pred = (nn_weight*nn_pred) + (svm_weight*svm_pred) + (lr_weight*lr_pred)
end_time = time.time()
time_taken = end_time - start_time
y_pred = (y_pred >= 0.5).astype(int)
y_pred = y_pred.ravel()

start_time_eval = time.time()
lr_pred_eval = lr.predict(eval_data_x_tfidf)
svm_pred_eval = svm.predict(eval_data_x_tfidf)
nn_pred_eval = nn.predict(eval_data_x_tfidf)
nn_pred_eval = nn_pred_eval.ravel()

y_pred_eval = (nn_weight*nn_pred_eval) + (svm_weight*svm_pred_eval) + (lr_weight*lr_pred_eval)
end_time_eval = time.time()
time_taken_eval = end_time_eval - start_time_eval
y_pred_eval = (y_pred_eval >= 0.5).astype(int)
y_pred_eval = y_pred_eval.ravel()

metric_report(y_test,y_pred,time_taken,len(y_test), eval_data_y_polarity, y_pred_eval, time_taken_eval, len(eval_data_y_polarity))

27/27 [==============================] - 0s 882us/step
Model Metrics with testing data
The accuracy of the Model is 0.8695904173106646
The F-Score of the Model is 0.8619107424042407
The Recall of the Model is 0.8695904173106646
The precision of the Model is 0.8717069613965021
The time taken for the Model prediction is 10.577749013900757 seconds
The number of records per second is 489.0
Model Metrics with evaluation data
The accuracy of the Model is 0.8479809976247031
The F-Score of the Model is 0.8339784532507322
The Recall of the Model is 0.8479809976247031
The precision of the Model is 0.854245758640058
The time taken for the Model prediction is 1.6730055809020996 seconds
The number of records per second is 503.0
